https://selenium-python.readthedocs.io/locating-elements.html

In [1]:
#!pip install selenium-wire
#!pip install mysql-connector-python
import requests
import pandas as pd
from bs4 import BeautifulSoup
from seleniumwire import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import datetime
import json
import time
import mysql.connector

In [2]:
url="https://www.eex.com/en/market-data/natural-gas/futures"
doc=requests.get(url, timeout=25)
#doc.text

# Web Scrapping hasta los datos

In [3]:
#https://chromedriver.chromium.org/downloads
#Abrir el link en modo desarrollador para poder hacer el scrapping.
url="https://www.eex.com/en/market-data/natural-gas/futures"
browser = webdriver.Chrome(executable_path=r"C:\Users\jlmerino\Downloads\chromedriver.exe")
browser.get(url)
time.sleep(4)

In [4]:
#Este elemento seria para hacer click a 'I ACCEPT ALL COOCKIES.'
#Justamente para poder acceder a él, hay un nombre de clase que solo usa el,
#así que busco por el nombre de esa clase y hago click.
aceptarCookies = browser.find_element(By.CLASS_NAME, 'uo_cookie_btn_type_0')
aceptarCookies.click()
time.sleep(4)

In [5]:
#Con esto abririamos el desplegable para luego poder seleccionar el elemento que queremos.
dropdown = browser.find_element(By.CLASS_NAME, 'filter-option-inner-inner')
dropdown.click()

In [6]:
#Seleccionamos el elemento que queremos del desplegable.
dropdownList = browser.find_element(By.XPATH, '//*[@id="mm-0"]/div[2]/main/div/div/div[1]/div/div/div/ul/li[13]/a')
dropdownList.click()

In [7]:
#Ponemos que muestre los valores en meses.
month = browser.find_element(By.XPATH, '//*[@id="symbolheader_ngfttf"]/div/div[2]/div[4]')
month.click()

# Web Scrapping Febrero

In [8]:
#Hacemos click en la mini imagen del calendario.
calendarioBueno = browser.find_element(By.ID, 'symbolheader_ngfttf')
calendarioBueno = calendarioBueno.find_element(By.CSS_SELECTOR, 'svg')
calendarioBueno.click()

In [9]:
#Tiramos hacia atras una vez para ver los días de Febrero.
for n in range(0,1):
    flechaIzquierda = browser.find_element(By.CLASS_NAME, 'mv-cal-prev-month')
    flechaIzquierda.click()

In [10]:
#Se guardan los elementos de ese mes.
calendario = browser.find_element(By.CLASS_NAME, 'mv-calendar')

In [11]:
#Se guardan los días que en los que se pueda hacer click.
dias = calendario.find_element(By.XPATH, '//*[@id="symbolheader_ngfttf"]/div/div[1]/div[2]/table/tbody')
diasMes = dias.find_elements(By.CSS_SELECTOR, ':not(.mv-cal-day.mv-disabled)')
diasPrimerMes = []
primerMesDias = {}
for n in diasMes:
    if n.get_attribute('class') == 'mv-button-base mv-hyperlink-button mv-cal-day':
        diasPrimerMes.append(n)
        primerMesDias[n.text] = n

In [12]:
primerMesDias.keys()

dict_keys(['9', '10', '13', '14', '15', '16', '17', '20', '21', '22', '23', '24', '27', '28'])

In [13]:
#Se mira por cada día en el que se puede hacer click de ese mes los valores de la cuarta columna de los valores y se guardan.
#A su vez, tambien se guarda una vez los valores de la primera columna de los valores.
#Luego se debe hacer el mismo proceso anterior de mirar los días disponibles ya que se ve que cuando hace siguiente vuelta
#da error por que ya no es accesible.

primerMesValores = {}
valoresFebrero = []
primero = True
primeroFuture = True

#Valores de días anteriores.
primerMesValores[6] = ['55,100', '58,011', '59,065', '59,345', '59,617', '59,863', '60,280']
primerMesValores[7] = ['55,100', '55,499', '56,481', '56,836', '57,243', '57,737', '58,129']
primerMesValores[8] = ['55,100', '53,637', '54,643', '55,082', '55,613', '56,076', '56,026']

for num, dia in primerMesDias.items():
    if primero:
        dia.click()
        time.sleep(3)
        primero = False
    
    tablaValores = browser.find_element(By.XPATH, '//*[@id="baseloadwidget_ngfttf"]/table')
    tablaValores = tablaValores.find_element(By.CSS_SELECTOR, 'tbody').find_elements(By.CSS_SELECTOR, 'tr')
    
    if primeroFuture:
        primeroFuture = False
        for row in tablaValores:
            valor = row.find_elements(By.CSS_SELECTOR, 'td')[0]
            valor = valor.text
            valoresFebrero.append(valor)
    
    valores = []
    for row in tablaValores:
        valor = row.find_elements(By.CSS_SELECTOR, 'td')[3]
        valor = valor.text
        valores.append(valor)
        
    primerMesValores[num] = valores
    print(primerMesValores[num])
    
    calendarioBueno.click()
    
    calendario = browser.find_element(By.CLASS_NAME, 'mv-calendar')
    dias = calendario.find_element(By.XPATH, '//*[@id="symbolheader_ngfttf"]/div/div[1]/div[2]/table/tbody')
    diasMes = dias.find_elements(By.CSS_SELECTOR, ':not(.mv-cal-day.mv-disabled)')
    primerMesDias = {}
    for n in diasMes:
        if n.get_attribute('class') == 'mv-button-base mv-hyperlink-button mv-cal-day':
            primerMesDias[n.text] = n
            
    if num == '10' or num == '17' or num == '24':
        primerMesDias[str(int(num) + 3)].click()
    else:
        if num == '28':
            break
        else:
            primerMesDias[str(int(num) + 1)].click()
    time.sleep(3)
    
    

['55,100', '52,804', '53,770', '54,252', '54,696', '54,967', '55,545']
['55,100', '53,741', '54,380', '54,804', '55,109', '55,403', '55,875']
['55,100', '52,025', '52,314', '52,667', '53,040', '53,228', '53,869']
['55,100', '52,400', '52,630', '52,850', '53,120', '53,102', '53,552']
['55,100', '54,666', '54,605', '54,739', '55,023', '55,202', '55,565']
['55,100', '52,030', '52,447', '52,775', '53,113', '53,017', '53,400']
['55,100', '49,100', '49,660', '50,110', '50,510', '50,780', '51,170']
['55,100', '49,924', '49,949', '50,038', '50,388', '50,645', '50,920']
['55,100', '48,658', '48,739', '48,874', '49,172', '49,462', '49,632']
['55,100', '50,569', '50,359', '50,340', '50,443', '50,682', '50,867']
['55,100', '50,852', '50,738', '50,832', '51,196', '51,261', '51,518']
['55,100', '51,050', '50,820', '50,990', '51,330', '51,560', '51,910']
['47,508', '47,564', '47,826', '48,209', '48,422', '48,733', '49,389']
['47,508', '46,720', '46,990', '47,420', '47,612', '47,872', '48,532']


In [14]:
primerMesValores

{6: ['55,100', '58,011', '59,065', '59,345', '59,617', '59,863', '60,280'],
 7: ['55,100', '55,499', '56,481', '56,836', '57,243', '57,737', '58,129'],
 8: ['55,100', '53,637', '54,643', '55,082', '55,613', '56,076', '56,026'],
 '9': ['55,100', '52,804', '53,770', '54,252', '54,696', '54,967', '55,545'],
 '10': ['55,100', '53,741', '54,380', '54,804', '55,109', '55,403', '55,875'],
 '13': ['55,100', '52,025', '52,314', '52,667', '53,040', '53,228', '53,869'],
 '14': ['55,100', '52,400', '52,630', '52,850', '53,120', '53,102', '53,552'],
 '15': ['55,100', '54,666', '54,605', '54,739', '55,023', '55,202', '55,565'],
 '16': ['55,100', '52,030', '52,447', '52,775', '53,113', '53,017', '53,400'],
 '17': ['55,100', '49,100', '49,660', '50,110', '50,510', '50,780', '51,170'],
 '20': ['55,100', '49,924', '49,949', '50,038', '50,388', '50,645', '50,920'],
 '21': ['55,100', '48,658', '48,739', '48,874', '49,172', '49,462', '49,632'],
 '22': ['55,100', '50,569', '50,359', '50,340', '50,443', '50,

In [15]:
#Se crea un dataframe con los valores.
dfFebrero = pd.DataFrame(
    data=primerMesValores.values(),
    columns=valoresFebrero,
    index=primerMesValores.keys()
)
dfFebrero

,Feb/23,Mar/23,Apr/23,May/23,Jun/23,Jul/23,Aug/23
6,"55,100","58,011","59,065","59,345","59,617","59,863","60,280"
7,"55,100","55,499","56,481","56,836","57,243","57,737","58,129"
8,"55,100","53,637","54,643","55,082","55,613","56,076","56,026"
9,"55,100","52,804","53,770","54,252","54,696","54,967","55,545"
10,"55,100","53,741","54,380","54,804","55,109","55,403","55,875"
13,"55,100","52,025","52,314","52,667","53,040","53,228","53,869"
14,"55,100","52,400","52,630","52,850","53,120","53,102","53,552"
15,"55,100","54,666","54,605","54,739","55,023","55,202","55,565"
16,"55,100","52,030","52,447","52,775","53,113","53,017","53,400"
17,"55,100","49,100","49,660","50,110","50,510","50,780","51,170"


# Web Scrapping Marzo

In [16]:
#Hacemos click en el boton para ir al mes siguiente.
flechaIzquierda = browser.find_element(By.CLASS_NAME, 'mv-cal-next-month')
flechaIzquierda.click()

In [17]:
#Volvemos a guardar los valores de ese mes.
calendario = browser.find_element(By.CLASS_NAME, 'mv-calendar')

In [18]:
#Se guardan los días en los que se puedan hacer click.
dias = calendario.find_element(By.XPATH, '//*[@id="symbolheader_ngfttf"]/div/div[1]/div[2]/table/tbody')
diasMes = dias.find_elements(By.CSS_SELECTOR, ':not(.mv-cal-day.mv-disabled)')
diasSegundoMes = []
segundoMesDias = {}
for n in diasMes:
    if n.get_attribute('class') == 'mv-button-base mv-hyperlink-button mv-cal-day':
        diasSegundoMes.append(n)
        segundoMesDias[n.text] = n

In [19]:
segundoMesDias.keys()

dict_keys(['1', '2', '3', '6', '7', '8', '9', '10', '13', '14', '15', '16', '17', '20', '21', '22', '23', '24'])

In [20]:
#Se vuelve a mirar por cada día en el que se puede hacer click de ese mes los valores de la cuarta columna de los valores y 
#se guardan. Tambien a su vez, se guarda una vez los valores de la primera columna de los valores.
#Luego se debe hacer el mismo proceso anterior de mirar los días disponibles ya que se ve que cuando hace siguiente vuelta
#da error por que ya no es accesible.

segundoMesValores = {}
valoresMarzo = []
primero = True
primeroFuture = True
for num, dia in segundoMesDias.items():
    if primero:
        dia.click()
        time.sleep(3)
        primero = False
    
    tablaValores = browser.find_element(By.XPATH, '//*[@id="baseloadwidget_ngfttf"]/table')
    tablaValores = tablaValores.find_element(By.CSS_SELECTOR, 'tbody').find_elements(By.CSS_SELECTOR, 'tr')
    
    if primeroFuture:
        primeroFuture = False
        for row in tablaValores:
            valor = row.find_elements(By.CSS_SELECTOR, 'td')[0]
            valor = valor.text
            valoresMarzo.append(valor)
    
    valores = []
    for row in tablaValores:
        valor = row.find_elements(By.CSS_SELECTOR, 'td')[3]
        valor = valor.text
        valores.append(valor)
        
    segundoMesValores[num] = valores
    print(segundoMesValores[num])
    
    calendarioBueno.click()
    
    calendario = browser.find_element(By.CLASS_NAME, 'mv-calendar')
    dias = calendario.find_element(By.XPATH, '//*[@id="symbolheader_ngfttf"]/div/div[1]/div[2]/table/tbody')
    diasMes = dias.find_elements(By.CSS_SELECTOR, ':not(.mv-cal-day.mv-disabled)')
    segundoMesDias = {}
    for n in diasMes:
        if n.get_attribute('class') == 'mv-button-base mv-hyperlink-button mv-cal-day':
            segundoMesDias[n.text] = n
            
    if num == '3' or num == '10' or num == '17':
        segundoMesDias[str(int(num) + 3)].click()
    else:
        if num == '24':
            break
        else:
            segundoMesDias[str(int(num) + 1)].click()
    time.sleep(3)
    
    

['47,508', '47,020', '47,140', '47,590', '47,790', '48,070', '48,820']
['47,508', '46,775', '46,927', '47,340', '47,442', '47,794', '48,466']
['47,508', '45,030', '45,150', '45,490', '45,583', '46,028', '46,708']
['47,508', '42,140', '42,420', '42,850', '43,086', '43,486', '44,096']
['47,508', '43,270', '43,400', '43,710', '44,006', '44,426', '45,086']
['47,508', '42,300', '42,705', '43,125', '43,250', '43,640', '44,400']
['47,508', '43,385', '43,625', '44,115', '44,322', '44,554', '45,300']
['47,508', '52,671', '52,807', '53,043', '52,431', '52,286', '54,564']
['47,508', '49,861', '50,119', '50,554', '50,860', '50,689', '51,137']
['47,508', '44,250', '44,720', '45,030', '45,140', '45,475', '46,287']
['47,508', '42,956', '43,371', '43,718', '44,057', '44,291', '45,020']
['47,508', '44,359', '44,783', '45,200', '45,383', '45,670', '46,475']
['47,508', '43,026', '43,419', '43,854', '43,976', '44,452', '45,183']
['47,508', '39,580', '40,055', '40,475', '40,642', '41,082', '41,952']
['47,5

In [21]:
segundoMesValores

{'1': ['47,508', '47,020', '47,140', '47,590', '47,790', '48,070', '48,820'],
 '2': ['47,508', '46,775', '46,927', '47,340', '47,442', '47,794', '48,466'],
 '3': ['47,508', '45,030', '45,150', '45,490', '45,583', '46,028', '46,708'],
 '6': ['47,508', '42,140', '42,420', '42,850', '43,086', '43,486', '44,096'],
 '7': ['47,508', '43,270', '43,400', '43,710', '44,006', '44,426', '45,086'],
 '8': ['47,508', '42,300', '42,705', '43,125', '43,250', '43,640', '44,400'],
 '9': ['47,508', '43,385', '43,625', '44,115', '44,322', '44,554', '45,300'],
 '10': ['47,508', '52,671', '52,807', '53,043', '52,431', '52,286', '54,564'],
 '13': ['47,508', '49,861', '50,119', '50,554', '50,860', '50,689', '51,137'],
 '14': ['47,508', '44,250', '44,720', '45,030', '45,140', '45,475', '46,287'],
 '15': ['47,508', '42,956', '43,371', '43,718', '44,057', '44,291', '45,020'],
 '16': ['47,508', '44,359', '44,783', '45,200', '45,383', '45,670', '46,475'],
 '17': ['47,508', '43,026', '43,419', '43,854', '43,976', '

In [22]:
#Se crea un dataframe con los valores.
dfMarzo = pd.DataFrame(
    data=segundoMesValores.values(),
    columns=valoresMarzo,
    index=segundoMesValores.keys()
)
dfMarzo

,Mar/23,Apr/23,May/23,Jun/23,Jul/23,Aug/23,Sep/23
1,"47,508","47,020","47,140","47,590","47,790","48,070","48,820"
2,"47,508","46,775","46,927","47,340","47,442","47,794","48,466"
3,"47,508","45,030","45,150","45,490","45,583","46,028","46,708"
6,"47,508","42,140","42,420","42,850","43,086","43,486","44,096"
7,"47,508","43,270","43,400","43,710","44,006","44,426","45,086"
8,"47,508","42,300","42,705","43,125","43,250","43,640","44,400"
9,"47,508","43,385","43,625","44,115","44,322","44,554","45,300"
10,"47,508","52,671","52,807","53,043","52,431","52,286","54,564"
13,"47,508","49,861","50,119","50,554","50,860","50,689","51,137"
14,"47,508","44,250","44,720","45,030","45,140","45,475","46,287"


# Subir los datos a SQL

In [23]:
dfFebrero = dfFebrero.reset_index()
dfFebrero = dfFebrero.rename(columns={'index': 'id'})
dfFebrero = dfFebrero.replace(',', '.', regex=True)
dfFebrero = dfFebrero.rename(columns=lambda x: x.replace('/', '_'))
dfFebrero

,id,Feb_23,Mar_23,Apr_23,May_23,Jun_23,Jul_23,Aug_23
0,6,55.100,58.011,59.065,59.345,59.617,59.863,60.280
1,7,55.100,55.499,56.481,56.836,57.243,57.737,58.129
2,8,55.100,53.637,54.643,55.082,55.613,56.076,56.026
3,9,55.100,52.804,53.770,54.252,54.696,54.967,55.545
4,10,55.100,53.741,54.380,54.804,55.109,55.403,55.875
5,13,55.100,52.025,52.314,52.667,53.040,53.228,53.869
6,14,55.100,52.400,52.630,52.850,53.120,53.102,53.552
7,15,55.100,54.666,54.605,54.739,55.023,55.202,55.565
8,16,55.100,52.030,52.447,52.775,53.113,53.017,53.400
9,17,55.100,49.100,49.660,50.110,50.510,50.780,51.170


In [24]:
dfMarzo = dfMarzo.reset_index()
dfMarzo = dfMarzo.rename(columns={'index': 'id'})
dfMarzo = dfMarzo.replace(',', '.', regex=True)
dfMarzo = dfMarzo.rename(columns=lambda x: x.replace('/', '_'))
dfMarzo

,id,Mar_23,Apr_23,May_23,Jun_23,Jul_23,Aug_23,Sep_23
0,1,47.508,47.020,47.140,47.590,47.790,48.070,48.820
1,2,47.508,46.775,46.927,47.340,47.442,47.794,48.466
2,3,47.508,45.030,45.150,45.490,45.583,46.028,46.708
3,6,47.508,42.140,42.420,42.850,43.086,43.486,44.096
4,7,47.508,43.270,43.400,43.710,44.006,44.426,45.086
5,8,47.508,42.300,42.705,43.125,43.250,43.640,44.400
6,9,47.508,43.385,43.625,44.115,44.322,44.554,45.300
7,10,47.508,52.671,52.807,53.043,52.431,52.286,54.564
8,13,47.508,49.861,50.119,50.554,50.860,50.689,51.137
9,14,47.508,44.250,44.720,45.030,45.140,45.475,46.287


In [25]:
mydb = mysql.connector.connect(
  host="192.168.193.133",
  user="sistemesbd",
  password="bigdata2223",
  database="alumnes"
)
mydb

In [26]:
#Crea la tabla de David_Febrero, como ya esta creada dicha tabla

cursor = mydb.cursor()
table_name = "David_Febrero"
create_table_query = """
CREATE TABLE IF NOT EXISTS {} (
  id INT NOT NULL,
  Feb_23 FLOAT,
  Mar_23 FLOAT,
  Apr_23 FLOAT,
  May_23 FLOAT,
  Jun_23 FLOAT,
  Jul_23 FLOAT,
  Aug_23 FLOAT,
  PRIMARY KEY (id)
);
""".format(table_name)
cursor.execute(create_table_query)

mydb.close()

In [27]:
mydb = mysql.connector.connect(
  host="192.168.193.133",
  user="sistemesbd",
  password="bigdata2223",
  database="alumnes"
)
mydb

In [28]:
mycursor = mydb.cursor()
for index, row in dfFebrero.iterrows():
    query = "INSERT INTO David_Febrero (id, Feb_23, Mar_23, Apr_23, May_23, Jun_23, Jul_23, Aug_23) VALUES (%s, %s, %s, %s, %s, %s, %s, %s) ON DUPLICATE KEY UPDATE Feb_23=VALUES(Feb_23), Mar_23=VALUES(Mar_23), Apr_23=VALUES(Apr_23), May_23=VALUES(May_23), Jun_23=VALUES(Jun_23), Jul_23=VALUES(Jul_23), Aug_23=VALUES(Aug_23)"
    values = (row['id'], row['Feb_23'], row['Mar_23'], row['Apr_23'], row['May_23'], row['Jun_23'], row['Jul_23'], row['Aug_23'])
    mycursor.execute(query, values)

#Confirma los cambios en la base de datos
mydb.commit()

mydb.close()

In [29]:
mydb = mysql.connector.connect(
  host="192.168.193.133",
  user="sistemesbd",
  password="bigdata2223",
  database="alumnes"
)
mydb

In [30]:
cursor = mydb.cursor()
table_name = "David_Marzo"
create_table_query = """
CREATE TABLE IF NOT EXISTS {} (
  id INT NOT NULL,
  Mar_23 FLOAT,
  Apr_23 FLOAT,
  May_23 FLOAT,
  Jun_23 FLOAT,
  Jul_23 FLOAT,
  Aug_23 FLOAT,
  Sep_23 FLOAT,
  PRIMARY KEY (id)
);
""".format(table_name)
cursor.execute(create_table_query)

mydb.close()

In [31]:
mydb = mysql.connector.connect(
  host="192.168.193.133",
  user="sistemesbd",
  password="bigdata2223",
  database="alumnes"
)
mydb

In [32]:
mycursor = mydb.cursor()
for index, row in dfMarzo.iterrows():
    query = "INSERT INTO David_Marzo (id, Mar_23, Apr_23, May_23, Jun_23, Jul_23, Aug_23, Sep_23) VALUES (%s, %s, %s, %s, %s, %s, %s, %s) ON DUPLICATE KEY UPDATE Mar_23=VALUES(Mar_23), Apr_23=VALUES(Apr_23), May_23=VALUES(May_23), Jun_23=VALUES(Jun_23), Jul_23=VALUES(Jul_23), Aug_23=VALUES(Aug_23), Sep_23=VALUES(Sep_23)"
    values = (row['id'], row['Mar_23'], row['Apr_23'], row['May_23'], row['Jun_23'], row['Jul_23'], row['Aug_23'], row['Sep_23'])
    mycursor.execute(query, values)

#Confirma los cambios en la base de datos
mydb.commit()

mydb.close()